# Example usage with Python 3
This notebook demonstrates usage of `petab_select` to perform forward selection in a Python 3 script.

## Problem setup with initial model

Dependencies are imported. A model selection problem is loaded from the specification files.

In [1]:
import petab_select
from petab_select import (
    Model,
    ForwardCandidateSpace,
)
selection_problem = petab_select.Problem.from_yaml('model_selection/selection_problem.yaml')
model_space = selection_problem.model_space

The initial model is setup. This is a virtual model in the sense that it may be an invalid model from the perspective of the modeller. The initial model has no estimated parameters.

In [2]:
parameters = {
    k: 0.0
    for k in model_space.parameter_ids
}

initial_virtual_model = Model(
    model_id='INITIAL_VIRTUAL_MODEL',
    petab_yaml='.',
    parameters=parameters,
)

## First iteration

Neighbors of the initial model in the model space are identified for testing.

The model candidate space is setup with the initial model. The model space is then used to find neighbors to the initial model. The candidate space is used to calculate distances between models, and whether a candidate model represents a valid move in model space.

The in-built `ForwardCandidateSpace` uses the following properties to identify candidate models:
- previously estimated parameters must not be fixed;
- the number of estimated parameters must increase; and
- the increase in the number of estimated parameters must be minimal.

The model space keeps a history of identified neighbors, such that subsequent calls ignore previously identified neighbors. This can be disabled by changing usage to `petab_select.ModelSpace.neighbors(..., exclude=False)`, or reset to forget all history with `petab_select.ModelSpace.reset()`.

In [3]:
candidate_space = petab_select.ForwardCandidateSpace(initial_virtual_model)
test_models = model_space.neighbors(candidate_space)

In [4]:
for test_model in test_models:
    print(test_model.model_id)

M1_1_0
M1_2_0
M1_3_0


Each of the test models includes information that should be sufficient for model calibration with any suitable tool that supports PEtab.

NB: the `petab_yaml_path` is for the original PEtab problem, and would need to be customized by `parameters` to be the actual test model. `nan` indicates that a parameter should be estimated.

In [5]:
print(
    f'Model ID: {test_model.model_id}\n'
    f'PEtab YAML path: {test_model.petab_yaml}\n'
    f'Custom model parameters: {test_model.parameters}'
)

Model ID: M1_3_0
PEtab YAML path: model_selection/example_modelSelection.yaml
Custom model parameters: {'k1': nan, 'k2': 0.1, 'k3': 0.0}


At this point, a model calibration tool is used to find the best of the test models, according to some criterion. PEtab select can select the best model from a collection of models that provide a value for this criterion, or a specific model can be supplied. Here, PEtab Select will be used to select the best model from multiple models. At the end of the following iterations, a specific model will be provided.

In [6]:
# Set fake criterion values that might be the output of a model calibration tool.
fake_criterion_values = {
    'M1_1_0': 100,
    'M1_2_0': 50,
    'M1_3_0': 150,
}
for model in test_models:
    model.set_criterion(selection_problem.criterion, fake_criterion_values[model.model_id])

In [7]:
chosen_model = selection_problem.get_best(test_models)
print(chosen_model.model_id)

M1_2_0


## Second iteration
The process then repeats.

The chosen model is used to reset the candidate space such that neighboring models are identified with respect to the chosen model.

In [8]:
candidate_space.reset(chosen_model)
test_models = model_space.neighbors(candidate_space)

In [9]:
for test_model in test_models:
    print(test_model.model_id)

M1_4_0
M1_6_0


This time, the model calibration tool selects `M1_6_0`.

In [10]:
chosen_model = test_models[1]
print(chosen_model.model_id)

M1_6_0


## Third iteration

In [11]:
candidate_space.reset(chosen_model)
test_models = model_space.neighbors(candidate_space)

In [12]:
for test_model in test_models:
    print(test_model.model_id)

M1_7_0


In [13]:
chosen_model = test_models[0]
print(chosen_model.model_id)

M1_7_0


## Fourth iteration
The `M1_7_0` model is the largest model in the model space, so no valid neighbors are identified for the forward selection method.

At this point, the results of the model calibration tool for the different models can be used to select the best model.

In [14]:
candidate_space.reset(chosen_model)
test_models = model_space.neighbors(candidate_space)

In [15]:
for test_model in test_models:
    print(test_model.model_id)

In [16]:
print(test_models)

[]


## Fifth iteration
Note that there exist additional, uncalibrated models in the model space, which can be identified with the brute-force method.

In [17]:
candidate_space = petab_select.BruteForceCandidateSpace()
test_models = model_space.neighbors(candidate_space)

In [18]:
for test_model in test_models:
    print(test_model.model_id)

M1_0_0
M1_5_0
